# Ground Truth Evaluation

In [1]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

test_queries = json.load(open('test_queries.json'))
embeddings = np.load('../2-embedding/embeddings_gbert.npy')
model = SentenceTransformer('deepset/gbert-large')

chunks = []

with open('../1-normalisation+chunking/specs_chunks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        chunks.append(json.loads(line))
with open('../1-normalisation+chunking/descs_chunks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        chunks.append(json.loads(line))

# >3 zeichen fix (siehe embedding, copy&paste)
import pandas as pd
specs_df = pd.read_json('../1-normalisation+chunking/specs_chunks.jsonl', lines=True)
descs_df = pd.read_json('../1-normalisation+chunking/descs_chunks.jsonl', lines=True)
all_chunks = pd.concat([specs_df, descs_df], ignore_index=True)
no_empty_mask = all_chunks['document'].str.len() > 3
all_chunks_clean = all_chunks[no_empty_mask].reset_index(drop=True)
chunks = all_chunks_clean.to_dict('records')


/home/bruce/.local/share/mise/installs/python/3.13.7/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.


**Queries durchgehen**

In [7]:
all_results = {}

for category, queries in test_queries['queries'].items():
    for query_item in tqdm(queries, desc=f"  {category} queries", leave=False):
        query_id = query_item['id']
        query_text = query_item['query']

        top_10_results = retrive(query_text, top_k=10)

        all_results[query_id] = {
            'query': query_text,
            'category': category,
            'results': top_10_results
        }

with open('./retrival_results.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)